In [1]:
load=[10,15,7]   ## for all the vnf i.e. 3
resource=[1,5,8,10,10,10,15]   ## for all the servers

In [2]:
## bandwidth matrix
serverCount=7
import random
bandwidth=[[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
for i in range(0,serverCount):
    for j in range(i+1,serverCount):
        x=random.randint(1,10)
        bandwidth[i][j]=x
        bandwidth[j][i]=x    
bandwidth

[[0, 3, 6, 2, 9, 1, 6],
 [3, 0, 5, 6, 8, 1, 4],
 [6, 5, 0, 10, 7, 3, 9],
 [2, 6, 10, 0, 8, 7, 10],
 [9, 8, 7, 8, 0, 9, 3],
 [1, 1, 3, 7, 9, 0, 3],
 [6, 4, 9, 10, 3, 3, 0]]

In [3]:
traffic=[[0,5,10],
         [7,0,2],
         [6,2,0]]

In [4]:
# matrix for latency between servers
serverCount =7
matrix=[[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
for i in range(0,serverCount):
    for j in range(i+1,serverCount):
        x=random.randint(1,10)
        matrix[i][j]=x
        matrix[j][i]=x    
matrix

[[0, 8, 1, 6, 6, 6, 7],
 [8, 0, 4, 1, 2, 9, 6],
 [1, 4, 0, 2, 6, 10, 10],
 [6, 1, 2, 0, 8, 9, 6],
 [6, 2, 6, 8, 0, 8, 7],
 [6, 9, 10, 9, 8, 0, 3],
 [7, 6, 10, 6, 7, 3, 0]]

In [5]:
# for bandwidth constraint

def bandwidthConstraint(x1,x2,x3):
    if(traffic[0][1]<=bandwidth[x1][x2] and traffic[1][2]<=bandwidth[x2][x3]):
        return True
    else:
        return False

In [6]:
## for load resource constraint
def load_resource(x1,x2,x3):
    if(load[0]<=resource[x1] and load[1]<=resource[x2]  and load[2]<=resource[x3]):
        return True
    else:
        return False

In [7]:
def objectiveFunction(x):
    total_latency=0
    ## fetch top 3 probailities for vnf placemnent
    firstmax=-1
    firstvnf=-1
    secondmax=-1
    secondvnf=-1
    thirdmax=-1
    thirdvnf=-1
    j=0
    for i in x:
        if i>firstmax:
            thirdmax=secondmax
            thirdvnf=secondvnf
            secondmax=firstmax
            secondvnf=firstvnf
            firstmax=i
            firstvnf=j
        elif i>secondmax:
            thirdmax=secondmax
            thirdvnf=secondvnf
            secondmax=i
            secondvnf=j
        elif i>thirdmax:
            thirdmax=i
            thirdvnf=j
        j=j+1
    
    total_latency=matrix[firstvnf][secondvnf]+matrix[secondvnf][thirdvnf]
    if(load_resource(firstvnf,secondvnf,thirdvnf)):
        if(bandwidthConstraint(firstvnf,secondvnf,thirdvnf)):
            print("three vnf",firstvnf,secondvnf,thirdvnf)
            print(total_latency)
            return total_latency
        else:
            print("error bandwidth")
            return 1000000
    else:
        print("error load")
        return 1000000

In [8]:
class Particle:
    def __init__(self):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual inshort it is best latency
        self.err_i=-1               # error individual
        
        
        for i in range(0,num_dimensions):      # intializing position and velocity
            self.position_i.append(random.uniform(0,1))
            self.velocity_i.append(random.uniform(-1,1))
            
            
    # evaluate current fitness
    def evaluate(self,objectiveFunction):
        self.err_i=objectiveFunction(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
        
    
            
     # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
                       
            # adjust maximum position if necessary
            #if self.position_i[i]>bounds[1]:
             #   self.position_i[i]=bounds[1]

            # adjust minimum position if neseccary
            #if self.position_i[i] < bounds[0]:
             #   self.position_i[i]=bounds[0]

     # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.7         # inertia weight 
        c1=1.5        # congative constant
        c2=1.5        # social constant
        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

In [9]:
class PSO:
    def __init__(self,objectiveFunction,serverCount,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=serverCount
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle())

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(objectiveFunction)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print (pos_best_g)
        print (err_best_g)

if __name__ == "__PSO__":
    main()


In [10]:
bounds=[0,1]
PSO(objectiveFunction,serverCount,bounds,num_particles=15,maxiter=70)

error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error bandwidth
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error bandwidth
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error load
error bandwidth
error load
error load
error load
error load
error load
error load
error 